In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Paso 1: Definir modelos Pydantic para salidas estructuradas

Estos modelos definen el **esquema** que los agentes devolverán. Usar `response_format` con Pydantic asegura:
- ✅ Extracción de datos con seguridad de tipos
- ✅ Validación automática
- ✅ Sin errores de análisis en respuestas de texto libre
- ✅ Enrutamiento condicional sencillo basado en campos


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Paso 2: Crear la Herramienta de Reservas de Hotel

Esta herramienta es la que el **availability_agent** utilizará para verificar si hay habitaciones disponibles. Usamos el decorador `@ai_function` para:
- Convertir una función de Python en una herramienta invocable por IA
- Generar automáticamente un esquema JSON para el LLM
- Manejar la validación de parámetros
- Permitir la invocación automática por agentes

Para esta demostración:
- **Estocolmo, Seattle, Tokio, Londres, Ámsterdam** → Hay habitaciones ✅
- **Todas las demás ciudades** → No hay habitaciones ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Paso 3: Definir funciones de condición para el enrutamiento

Estas funciones inspeccionan la respuesta del agente y determinan qué camino tomar en el flujo de trabajo.

**Patrón clave:**
1. Verificar si el mensaje es `AgentExecutorResponse`
2. Analizar la salida estructurada (modelo Pydantic)
3. Devolver `True` o `False` para controlar el enrutamiento

El flujo de trabajo evaluará estas condiciones en **los bordes** para decidir qué ejecutor invocar a continuación.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Paso 4: Crear un Ejecutor de Visualización Personalizado

Los ejecutores son componentes del flujo de trabajo que realizan transformaciones o efectos secundarios. Usamos el decorador `@executor` para crear un ejecutor personalizado que muestre el resultado final.

**Conceptos Clave:**
- `@executor(id="...")` - Registra una función como ejecutor de flujo de trabajo
- `WorkflowContext[Never, str]` - Indicaciones de tipo para entrada/salida
- `ctx.yield_output(...)` - Produce el resultado final del flujo de trabajo


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Paso 5: Cargar Variables de Entorno

Configura el cliente LLM. Este ejemplo funciona con:
- **Modelos de GitHub** (Nivel gratuito con token de GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Paso 6: Crear agentes de IA con salidas estructuradas

Creamos **tres agentes especializados**, cada uno envuelto en un `AgentExecutor`:

1. **availability_agent** - Verifica la disponibilidad de hoteles utilizando la herramienta
2. **alternative_agent** - Sugiere ciudades alternativas (cuando no hay habitaciones)
3. **booking_agent** - Fomenta la reserva (cuando hay habitaciones disponibles)

**Características clave:**
- `tools=[hotel_booking]` - Proporciona la herramienta al agente
- `response_format=PydanticModel` - Obliga a generar salidas JSON estructuradas
- `AgentExecutor(..., id="...")` - Envuelve al agente para su uso en el flujo de trabajo


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Paso 7: Construir el flujo de trabajo con bordes condicionales

Ahora utilizamos `WorkflowBuilder` para construir el gráfico con enrutamiento condicional:

**Estructura del flujo de trabajo:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Métodos clave:**
- `.set_start_executor(...)` - Establece el punto de entrada
- `.add_edge(from, to, condition=...)` - Agrega un borde condicional
- `.build()` - Finaliza el flujo de trabajo


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Puntos Clave y Próximos Pasos

### ✅ Lo que Has Aprendido:

1. **Patrón WorkflowBuilder**
   - Usa `.set_start_executor()` para definir el punto de entrada
   - Usa `.add_edge(from, to, condition=...)` para el enrutamiento condicional
   - Llama a `.build()` para finalizar el flujo de trabajo

2. **Enrutamiento Condicional**
   - Las funciones de condición inspeccionan `AgentExecutorResponse`
   - Analizan salidas estructuradas para tomar decisiones de enrutamiento
   - Devuelven `True` para activar un enlace, `False` para omitirlo

3. **Integración de Herramientas**
   - Usa `@ai_function` para convertir funciones de Python en herramientas de IA
   - Los agentes llaman automáticamente a las herramientas cuando es necesario
   - Las herramientas devuelven JSON que los agentes pueden analizar

4. **Salidas Estructuradas**
   - Usa modelos Pydantic para extracción de datos con tipos seguros
   - Configura `response_format=MyModel` al crear agentes
   - Analiza respuestas con `Model.model_validate_json()`

5. **Ejecutores Personalizados**
   - Usa `@executor(id="...")` para crear componentes de flujo de trabajo
   - Los ejecutores pueden transformar datos o realizar efectos secundarios
   - Usa `ctx.yield_output()` para producir resultados del flujo de trabajo

### 🚀 Aplicaciones en el Mundo Real:

- **Reservas de Viajes**: Verificar disponibilidad, sugerir alternativas, comparar opciones
- **Atención al Cliente**: Enrutar según el tipo de problema, sentimiento, prioridad
- **E-commerce**: Verificar inventario, sugerir alternativas, procesar pedidos
- **Moderación de Contenido**: Enrutar según puntuaciones de toxicidad, reportes de usuarios
- **Flujos de Aprobación**: Enrutar según monto, rol del usuario, nivel de riesgo
- **Procesamiento Multi-etapas**: Enrutar según calidad de datos, completitud

### 📚 Próximos Pasos:

- Agregar condiciones más complejas (criterios múltiples)
- Implementar bucles con gestión de estado del flujo de trabajo
- Añadir sub-flujos para componentes reutilizables
- Integrar con APIs reales (reservas de hoteles, sistemas de inventario)
- Añadir manejo de errores y rutas de respaldo
- Visualizar flujos de trabajo con las herramientas de visualización integradas



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
